<a href="https://colab.research.google.com/github/HarryBurgin/TSE-Assignment3/blob/main/TSE_Group_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set up**<br>
Import all relevant libraries - TensorFlow

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## Used to read csv data file
from google.colab import files
import csv
import re

Initiate dataset

In [49]:
dataset = []
smallDataReview = []
smallDataSent = []

**Create data tensors** <br>
Load csv file and clean data

In [24]:
#Function to create a list of reviews and a list of sentiments (+ve/-ve)
def listCreation(dataset):
    #Opens the csv file
    fileName = open('IMDB Dataset.csv', 'r', encoding='utf-8')
    file = csv.DictReader(fileName)
    #Iterates each row then appends dataset
    for col in file:
        dataset.append([col['review'],col['sentiment']])


def dataCleanup(dataset):
    #Iterates through the dataset
    for i in range (len(dataset)):
        for j in range (len(dataset[i])):
            #If current element is a review, not a sentiment
            if j == 0:
                #Replaces any text between <> with a blank
                dataset[i][j] = re.sub('<.*?>', '', dataset[i][j])

In [51]:
listCreation(dataset)
dataCleanup(dataset)

## Makes subsection of whole dataset for prototyping purposes
for index in range(10):
  smallDataReview.append(dataset[index][0])
  smallDataSent.append(dataset[index][1])

print(smallDataReview)
print(smallDataSent)


["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows w

**Create model**

In [8]:
model = keras.Sequential()
model.add(keras.Input(shape=(None, 28)))
model.add(layers.SimpleRNN(512, return_sequences=True, activation='relu'))
model.add(layers.SimpleRNN(512, activation='relu'))
model.add(layers.Dense(10))

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, None, 512)         276992    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 806,922
Trainable params: 806,922
Non-trainable params: 0
_________________________________________________________________
None


**Compile model**

In [7]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ["accuracy"]
    )